In [2]:
#Corrected code 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import qutip as qt
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib ipympl

SIGMA_X = qt.sigmax()
SIGMA_Z = qt.sigmaz()

# Global variable to track the active animation
active_animation = None

def simulate_spin_dynamics(qubit_type, delta, beta, t_start, t_end):
    global active_animation

    if qubit_type != 'Single spin':
        raise ValueError(f"Sorry, only 'Single spin' is supported for now. You entered: {qubit_type}")

    if t_end <= t_start:
        raise ValueError("End time must be greater than start time!")

    # Hamiltonian of the system
    h_static = (delta / 2) * SIGMA_X  
    
    # Environment Hamiltonian(its mentioned that this hamiltonian must impose error on quantum system,so i use the same beta to define the collapse operator)
    H_Environment =(beta/2)* SIGMA_Z
    H_total= h_static + H_Environment


    c_ops=[np.sqrt(beta)*SIGMA_Z]

    # Initial state: |0⟩
    initial_state = qt.basis(2, 0)
    
    # Time array
    times = np.linspace(t_start, t_end, 300)

    # Solve master equation
    result = qt.mesolve(H_total, initial_state, times,c_ops, [SIGMA_Z])
    sigma_z_expect = result.expect[0]

    # Analytic solution: Oscillation (Δ) + Decay (β)
    sz_analytic = np.cos(delta * times) * np.exp(-beta * times)

    # Plotting setup
    fig, (ax_static, ax_anim) = plt.subplots(1, 2, figsize=(15, 6))

    # Static plot
    ax_static.plot(times, sigma_z_expect, label='Numerical', color='darkred')
    ax_static.plot(times, sz_analytic, '--', label='Analytic', color='green')
    ax_static.set_xlabel('Time (t)')
    ax_static.set_ylabel(r'$\langle \sigma_z \rangle$')
    ax_static.set_title('📊 Static Qubit Evolution')
    ax_static.legend()
    ax_static.grid(True)

    # Animation plot
    ax_anim.set_xlim(t_start, t_end)
    ax_anim.set_ylim(np.min(sigma_z_expect) - 0.1, np.max(sigma_z_expect) + 0.1)
    ax_anim.set_xlabel('Time (t)')
    ax_anim.set_ylabel(r'$\langle \sigma_z \rangle$')
    ax_anim.set_title('🎬 Live Evolution Animation')
    anim_line, = ax_anim.plot([], [], lw=2, color='navy', label=r'$\langle \sigma_z \rangle$')
    ax_anim.legend()
    ax_anim.grid(True)

    # Animation functions
    def init_anim():
        anim_line.set_data([], [])
        return anim_line,

    def update_anim(frame):
        anim_line.set_data(times[:frame], sigma_z_expect[:frame])
        return anim_line,

    # Start animation
    print("🚀 Starting the animation... ")
    active_animation = animation.FuncAnimation(
        fig, update_anim, frames=len(times),
        init_func=init_anim, blit=True, interval=30
    )

    plt.tight_layout()
    plt.show()

    return active_animation

# Interactive widgets
interact(
    simulate_spin_dynamics,
    qubit_type=Dropdown(options=['Single spin'], value='Single spin', description='Qubit Type:'),
    delta=FloatSlider(value=2*np.pi, min=0.1, max=10.0, step=0.1, description='Δ (Delta):'),
    beta=FloatSlider(value=0.5, min=0.0, max=5.0, step=0.1, description='β (Beta):'),
    t_start=FloatSlider(value=0.0, min=0.0, max=10.0, step=0.1, description='Start Time:'),
    t_end=FloatSlider(value=5.0, min=1.0, max=20.0, step=0.1, description='End Time:')
);


interactive(children=(Dropdown(description='Qubit Type:', options=('Single spin',), value='Single spin'), Floa…